In [12]:
from google.colab import drive
drive.mount('/drive/')

Drive already mounted at /drive/; to attempt to forcibly remount, call drive.mount("/drive/", force_remount=True).


In [0]:
from keras import models, layers
from keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import math
 
 
 
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
 
train_dir = os.path.join('/drive/My Drive/Colab Notebooks/train')
val_dir = os.path.join('/drive/My Drive/Colab Notebooks/valid')
test_dir = os.path.join('/drive/My Drive/Colab Notebooks/test')

In [52]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=10, target_size=(220, 200), color_mode='rgb',class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, batch_size=10, target_size=(220, 200), color_mode='rgb',class_mode='categorical')
 
input_tensor = Input(shape=(220, 200, 3), dtype='float32', name='input')


Found 274 images belonging to 6 classes.
Found 49 images belonging to 6 classes.


In [53]:
pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(220, 200, 3))
pre_trained_vgg.trainable = False
pre_trained_vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 220, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 220, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 220, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 110, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 110, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 110, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 55, 50, 128)       0         
__________

In [30]:
additional_model = models.Sequential()
additional_model.add(pre_trained_vgg)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(4096, activation='relu'))
additional_model.add(layers.Dense(2048, activation='relu'))
additional_model.add(layers.Dense(1024, activation='relu'))
additional_model.add(layers.Dense(5, activation='softmax'))
 
 
additional_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 4096)              75501568  
_________________________________________________________________
dense_14 (Dense)             (None, 2048)              8390656   
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 6150      
Total params: 100,711,238
Trainable params: 85,996,550
Non-trainable params: 14,714,688
______________________________________________________

In [31]:
checkpoint = ModelCheckpoint(filepath='pretrained_VGG_weight.hdf5', 
            monitor='loss', 
            mode='min', 
            save_best_only=True)
 
additional_model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])
 
 
history = additional_model.fit_generator(train_generator, 
            steps_per_epoch=math.ceil(train_generator.n / train_generator.batch_size), 
            epochs=100, 
            validation_data=val_generator, 
            validation_steps=math.ceil(val_generator.n / val_generator.batch_size), 
            callbacks=[checkpoint])

Epoch 1/100
28/28 [==============================] - 12s 430ms/step - loss: 1.2317 - acc: 0.5234 - val_loss: 1.3252 - val_acc: 0.4286
Epoch 2/100
28/28 [==============================] - 11s 387ms/step - loss: 0.7306 - acc: 0.7160 - val_loss: 1.1755 - val_acc: 0.3878
Epoch 3/100
28/28 [==============================] - 11s 386ms/step - loss: 0.6100 - acc: 0.7716 - val_loss: 1.1281 - val_acc: 0.6735
Epoch 4/100
28/28 [==============================] - 12s 418ms/step - loss: 0.4693 - acc: 0.8213 - val_loss: 1.1107 - val_acc: 0.5714
Epoch 5/100
28/28 [==============================] - 12s 419ms/step - loss: 0.3770 - acc: 0.8646 - val_loss: 0.7255 - val_acc: 0.7143
Epoch 6/100
28/28 [==============================] - 11s 406ms/step - loss: 0.3661 - acc: 0.8606 - val_loss: 0.7606 - val_acc: 0.7143
Epoch 7/100
28/28 [==============================] - 12s 422ms/step - loss: 0.3258 - acc: 0.8749 - val_loss: 1.3857 - val_acc: 0.6122
Epoch 8/100
28/28 [==============================] - 11s 410ms

In [0]:
from keras.models import load_model

additional_model.save('model.h5')
additional_model.save_weights("model.h5")

model_json = additional_model.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)

In [68]:
print("-- Evaluate --")
scores = additional_model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(additional_model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 91.30%
